# Delta Radiomics

### **Import Libraries**

In [ ]:
import pandas as pd
import numpy as np
import os
import yaml

### **Import Data**

In [ ]:
with open("D:/DSLS/Omics2/integromics/config.yaml", "r") as f:
    config = yaml.safe_load(f)

print(config)


In [ ]:

def calculate_delta_radiomics(data_folder_path):
    """
    Reads radiomics data from subfolders (Time A and Time B), filters for 'suv2.5' 
    segmentation, calculates the delta (B - A) for numeric features, and stores
    the results in a dictionary per patient.

    Args:
        data_folder_path (str): The path to the main folder containing patient subfolders.

    Returns:
        dict: A dictionary where keys are patient folder names (Patient IDs) 
              and values are dictionaries containing the calculated delta radiomics features.
    """
    all_delta_radiomics = {}
    A_radiomics, B_radiomics = {}, {}

    # 1. Iterate through all items in the main data folder
    for patient_folder_name in os.listdir(data_folder_path):
        patient_path = os.path.join(data_folder_path, patient_folder_name)
        
        # Ensure it is actually a directory (a patient folder)
        if os.path.isdir(patient_path):
            print(f"--- Processing {patient_folder_name} ---")
            
            # Initialize paths for Time A and Time B files
            file_A_path = None
            file_B_path = None
            
            # 2. Find the radiomics files for Time A and Time B in the patient folder
            for filename in os.listdir(patient_path):
                path_excel = os.path.join(patient_path, filename)
                # NOTE: Assuming the files are named consistently and contain 'A' or 'B' 
                # to identify the time point. Adjust this logic if needed.
  
                if '_A' in path_excel.upper() and path_excel.endswith('.xlsx'):
                        file_A_path = path_excel
                elif '_B' in path_excel.upper() and path_excel.endswith('.xlsx'):
                        file_B_path = path_excel
            if file_A_path and file_B_path:
                try:
                    # 3. Read and preprocess the data
                    
                    # Read Excel files and transpose them (assuming features are in columns 
                    # and metadata/values in rows; pandas reads the first row as header)
                    # We assume 'segmentation' is one of the columns after reading.
                    df_A = pd.read_excel(file_A_path)
                    df_B = pd.read_excel(file_B_path)
                    
                    # 4. Filter for the 'suv2.5' segmentation row
                    # NOTE: the column containing 'suv2.5' is named 'Segmentation'
                    # and the feature names are in the other columns.
                    # filtering the columns fro 23 onwards to get only feature values
                    row_A = df_A[df_A['Segmentation'].str.contains('suv2.5')].iloc[0, 23:]
                    row_B = df_B[df_B['Segmentation'].str.contains('suv2.5')].iloc[0, 23:]

                    # Create a Series of only the numeric feature values for A and B
                    
                    # Convert to numeric, coercing errors to NaN (just in case)
                    numeric_A = pd.to_numeric(row_A, errors='coerce')
                    numeric_B = pd.to_numeric(row_B, errors='coerce')

                    # 6. Calculate Delta Radiomics (Time B - Time A)
                    delta_radiomics = numeric_B - numeric_A
                    
                    
                    # Convert the resulting pandas Series into a standard Python dictionary
                    # and store it under the patient's ID
                    # dropna() to remove any features that resulted in NaN
                    all_delta_radiomics[patient_folder_name] = delta_radiomics.dropna().to_dict()
                    A_radiomics[patient_folder_name] = numeric_A.dropna().to_dict()
                    B_radiomics[patient_folder_name] = numeric_B.dropna().to_dict()
                    print(f"Successfully calculated radiomics and delta radiomics for {patient_folder_name}.")

                except Exception as e:
                    print(f"Error processing files for {patient_folder_name}: {e}")
            else:
                print(f"Could not find both A and B files in {patient_folder_name}.")
    A = pd.DataFrame.from_dict(A_radiomics, orient='index')
    B = pd.DataFrame.from_dict(B_radiomics, orient='index')
    all_delta_radiomics = pd.DataFrame.from_dict(all_delta_radiomics, orient='index')

    return all_delta_radiomics, A, B


In [ ]:
# Define the path to main data folder

DATA_DIR = config["data"]["root_dir"]

# Run the function
delta_radiomics_results, a_radiomics, b_radiomics = calculate_delta_radiomics(DATA_DIR)

# Print the results for verification
print("\n--- Final Results Summary ---")
for patient, delta_data in delta_radiomics_results.items():
    print(f"\n{patient} Delta Radiomics ({len(delta_data)} features):")
    # Print the first 5 features as an example
    print(dict(list(delta_data.items())[:5]))

In [ ]:
delta_radiomics_results

In [ ]:
# Clean and prepare dataframes
# by dropping columns with any NaN values and resetting index
# to keep only the complete cases (some patients have 99 columns with NaNs, but 43 are always present)
# we'll work with those 43.
for df in [delta_radiomics_results, a_radiomics, b_radiomics]:
    df.dropna(axis=1, how='any', inplace=True)
    df.reset_index(inplace=True)
    df.rename(columns={'index': 'id'}, inplace=True)
    df['id'] = df['id'].astype(int)

In [ ]:
# to differentiate the columns of A and B datasets
a_radiomics = a_radiomics.add_suffix('_a')

In [ ]:
print(a_radiomics.head())


In [ ]:
b_radiomics = b_radiomics.add_suffix('_b')

In [ ]:
print(b_radiomics.head())

In [ ]:
for patient, delta_data in delta_radiomics_results.items():
    if len(delta_data) == 99:
        print(patient)

In [ ]:
filtered_results = {patient: data for patient, data in delta_radiomics_results.items() if len(data) != 99}

In [ ]:
len(filtered_results)

In [ ]:
for patient, delta_data in delta_radiomics_results.items():
        print(patient)

Other than Kylies folders, we're missing these:  
**12: missing folder A**  
**19: missing results for A**  
**21: missing folder B**  
**64: missing results for B**  

Therefore we're left with only 14 complete delta radiomics feature sets.

# Clinical Data

In [ ]:

clinical_dir = config["clinical"]["root_dir"]      # "D:/DSLS/Omics2/modelling/clinical_data"
clinical_file = config["clinical"]["main_file"]    # "10162025_UMCG_wide_export_Yescarta_infused_for_tFL_study.xlsx"

clinical_path = os.path.join(clinical_dir, clinical_file)
clinic_data = pd.read_excel(clinical_path)


In [ ]:
clinic_data.head()

In [ ]:
clinic_data

In [ ]:
clinic_data.shape

In [ ]:
clinic_data['record_id'].values

In [ ]:
clinic_data['id_cleaned'] = [value[-3:] for value in clinic_data['record_id'].values]

In [ ]:
clinic_data.head()

In [ ]:
clinic_data

In [ ]:
clinic_data['id_cleaned'].values

In [ ]:
delta_radiomics_results['id']

In [ ]:
patient_ids = clinic_data['id_cleaned'].values[1:].astype(int)

In [ ]:
# find patients that are in both datasets
# values starts from 1 to skip the comment row
intercept = [id for id in delta_radiomics_results['id'] if id in patient_ids]

In [ ]:
clinic_data['id_cleaned'] = ['ID'] + patient_ids.tolist()

In [ ]:
clinic_data_cleaned = clinic_data[clinic_data['id_cleaned'].isin(intercept)]

In [ ]:
clinic_data_cleaned.reset_index(drop=True, inplace=True)

In [ ]:
clinic_data_cleaned.shape

Eventually, we have 24 patients with complete clinical and delta radiomics data to work with.

**Note:** patient 95 is missing their clinical data. 

In [ ]:
# we now should select features we need for modelling the baseline, without the delta radiomics
clinic_data_cleaned

In [ ]:
# dropping columns with all NaN values
clinic_data_cleaned = clinic_data_cleaned.dropna(axis=1, how='all')

In [ ]:
clinic_data_cleaned.shape

In [ ]:
# we don't need factor columns for modelling as they are encoded already
factors = [factor for factor in clinic_data_cleaned.columns if 'factor' in factor]

In [ ]:
comments = [comm for comm in clinic_data_cleaned.columns if 'comment' in comm]

In [ ]:
comments

In [ ]:
locations = [loc for loc in clinic_data_cleaned.columns if 'loc' in loc]

In [ ]:
locations

In [ ]:
# these are highly correlated features with bmi
correlated = ['scr_height', 'scr_weight']

* scr_age (continuous) correlates to indication_age_60 (binary), we Keep scr_age (continuous). It retains more information and doesn't arbitrarily cut at 60.  
* indication_ldh_uln: we have the exact value for ldh  
* indication_extran_sites, indication_extran_invol, indication_extranodal_nr	These are highly related. we keep indication_extranodal_nr (exact number). It is the most granular quantitative measure.

In [ ]:
indicators = ['indication_ldh_uln','indication_age_60','indication_extran_sites', 'indication_extran_invol']

In [ ]:
# cause of death columns are not needed
cause_of_death = [cause for cause in clinic_data_cleaned.columns if '_cause' in cause]

In [ ]:
cause_of_death

**NOTE:** indication_dis_diagnosis must be one-hot encoded. as the disease is a nominal categorical feature.

In [ ]:
disease = pd.get_dummies(clinic_data_cleaned['indication_dis_diagnosis.factor']).astype(int)

In [ ]:
disease

In [ ]:
drop_columns = cause_of_death + factors + ['record_id','scr_date_tb1stmeeting', 'indication_dis_diagnosis'] + comments + locations + correlated + indicators
clinic_data_cleaned.drop(columns=drop_columns,inplace=True)

In [ ]:
clinic_data_cleaned.shape

In [ ]:
clinic_data_cleaned = pd.concat([clinic_data_cleaned, disease], axis=1)

In [ ]:
clinic_data_cleaned

In [ ]:
clinic_data_cleaned.replace({'NE': np.nan}, inplace=True)

In [ ]:
clinic_data_cleaned.describe()

In [ ]:
nans = clinic_data_cleaned.isna().sum().sort_values(ascending=False)

In [ ]:
# columns with more than 12 nans, which is half the data for the patients we have
nans[nans > 12]

In [ ]:
drop_nans = nans[nans > 12].index

In [ ]:
clinic_data_cleaned = clinic_data_cleaned.drop(columns=drop_nans)

In [ ]:
clinic_data_cleaned.shape

In [ ]:
clinic_data_cleaned.select_dtypes(include=['object']).columns

In [ ]:
clinic_data_cleaned.dtypes

In [ ]:
clinic_data_cleaned.columns

In [ ]:
# Assuming clinic_data_filtered is the DataFrame you want to convert
date_columns = [date for date in clinic_data_cleaned.columns if ('date' in date) or ('start' in date) or ('stop' in date)]
# 1. Use convert_dtypes() for general automatic inference
# This function automatically converts to best possible dtypes (e.g., object to string, int64 to Int64, float64 to Float64)
# It's particularly useful for handling missing values using pandas' nullable dtypes (e.g., pd.NA).
print("Applying general type conversion...")

# 2. Force remaining object columns that look like numbers to numeric
for col in clinic_data_cleaned.columns:
        if col not in date_columns:
            # Attempt to convert to numeric.
            # this is to fix a typo in columns where , is used instead of .
            if clinic_data_cleaned[col].dtype == 'object':
                clinic_data_cleaned[col] = pd.to_numeric(clinic_data_cleaned[col].str.replace(',','.'), errors='raise')
            print(f"  Converted column '{col}' to numeric.")
        else: 
            clinic_data_cleaned[col] = pd.to_datetime(clinic_data_cleaned[col], errors='coerce')
            print(f"  Converted column '{col}' to datetime.")
        
print("\nAutomatic type conversion complete.")

In [ ]:
clinic_data_cleaned.dtypes

In [ ]:
variances = clinic_data_cleaned.select_dtypes(include=np.number).var().sort_values()

In [ ]:
# zero variance columns are not useful for modelling so I am dropping them
zero_var = variances[variances == 0].index

In [ ]:
zero_var

In [ ]:
clinic_data_cleaned = clinic_data_cleaned.drop(columns=zero_var)

In [ ]:
clinic_data_cleaned.shape

In [ ]:
clinic_data_cleaned.head()

CRS and ICANS are two of the most significant and potentially life-threatening side effects associated with certain powerful immunotherapies, most notably CAR T-cell therapy (Chimeric Antigen Receptor T-cell therapy).


💥 1. Cytokine Release Syndrome (CRS)
CRS is the more common of the two toxicities and is essentially a massive, systemic inflammatory response.

What it is: When the modified CAR T-cells successfully attack cancer cells, they rapidly multiply and release large amounts of signaling molecules called cytokines into the bloodstream (hence the name "Cytokine Release Syndrome"). This rapid, massive release causes a widespread inflammatory state.

Symptoms: CRS symptoms resemble a severe flu:

High fever (the hallmark symptom)

Chills, muscle aches, and fatigue

Hypotension (low blood pressure)

Hypoxia (low oxygen/trouble breathing)

Fast heart rate (tachycardia)

In severe cases, it can lead to multi-organ failure.

The severity of CRS (often graded 1 to 4/5 by consensus guidelines like the ASTCT criteria) is a crucial prognostic factor.

🧠 2. Immune Effector Cell-Associated Neurotoxicity Syndrome (ICANS)
ICANS is a syndrome involving neurological damage or dysfunction caused by the same immune activation that triggers CRS.

What it is: It is a neurological toxicity that typically occurs after or concurrently with CRS. The exact mechanism is complex but involves the massive cytokine release and the T-cells themselves affecting the central nervous system.

Symptoms: ICANS symptoms can range from mild to life-threatening:

Confusion or disorientation

Aphasia (difficulty speaking or understanding language)

Impaired attention

Headache, tremors, or loss of coordination

In severe cases, it can cause seizures or cerebral edema (brain swelling).

ICANS severity is also graded using specific neurological assessment scores and is a very strong independent predictor of outcomes and morbidity after CAR T-cell therapy.

In [ ]:
clinic_data_cleaned.shape

In [ ]:
clinic_data_cleaned.columns

In [ ]:
# Impute missing values with the median for numeric columns
for col in clinic_data_cleaned.select_dtypes(include=np.number).columns:
    median_value = clinic_data_cleaned[col].median()
    clinic_data_cleaned[col].fillna(median_value, inplace=True)

In [ ]:
clinic_data_cleaned.isna().sum().sort_values(ascending=False)

In [ ]:
# there are date related column that still have nans, but we will not use them for modelling as we can't impute them easily
# also cli_st_lab_date is not needed
date_columns = [
    'indication_ind_date',
    'tr_car_inf_adm_date',
    'tr_car_ld_start',
    'tr_car_inf_date',
    'tr_car_inf_discharge_date',
    'ae_summ_start_date_v2',
    'ae_summ_crs_start_v2',
    'ae_summ_crs_stop_v2',
    'surv_prog_date',
    'surv_date',
    'cli_st_lab_date'
]

clinic_data_cleaned.drop(columns=date_columns, inplace=True)

In [ ]:
clinic_data_cleaned.shape

In [ ]:
clinic_data_cleaned.isna().sum().sum() # confirming no nans remain

In [ ]:
clinic_data_cleaned.columns

----

## Calculate Time-to-Event (T) to prepare for Cox Regression

We need to select one pair of dates to calculate the duration of follow-up (Time, or $T$).  
Choice: The standard time point for post-therapy outcomes is from the date of infusion to the date of follow-up/death.  
Start Date: tr_car_inf_date (Date of CAR-T infusion)  
End Date: surv_date (Date of last follow-up or death)  

In [ ]:
# # Calculate the Time-to-Event (T) in days
# # This is the time from infusion until the event (or censoring)
# clinic_data_cleaned['T'] = (
#     clinic_data_cleaned['surv_date'] - clinic_data_cleaned['tr_car_inf_date']
# ).dt.days

In [ ]:
clinic_data_cleaned['T']

## Define the Event Indicator (E)

We need a binary variable (Event, or $E$) that indicates if the event of interest occurred.Choice: The most common target is Overall Survival (OS), where the event is death.Event Variable: surv_status (Assuming this is a 0/1 indicator where 1 = death/event)

In [ ]:
# in the dataset, it's the opposite of what we want: 1 means event occurred (death), 0 means censored (alive)
# so we need to invert it
clinic_data_cleaned['surv_status'] = 1 - clinic_data_cleaned['surv_status']

In [ ]:
# rename surv_status to E for event
clinic_data_cleaned.rename(columns={'surv_status': 'E'}, inplace=True)

In [ ]:
clinic_data_cleaned['E']

In [ ]:
# we should separate the possible target variables for modelling now
target_variables = ['surv_time_bestresponse_car', 'surv_prog_after_car']
date_related = ['tr_car_inf_adm_date','tr_car_ld_start', 'tr_car_inf_date', 'tr_car_inf_discharge_date',
       'ae_summ_start_date_v2','surv_date','indication_ind_date']

In [ ]:
modelling_data = pd.concat([clinic_data_cleaned, delta_radiomics_results, a_radiomics, b_radiomics], axis=1)

In [ ]:
modelling_data.head()

In [ ]:
modelling_data.shape

In [ ]:
# we need to drop the last row, as the patient's clinical data is not available
modelling_data = modelling_data.iloc[:-1,:]

In [ ]:
# to find the baseline performance, we don't need id_cleaned as we're not going to
# use this column for adding the delta radiomics yet
X = modelling_data.drop(columns=target_variables + date_related + ['id_a','id_b','id_cleaned','id'])                    

In [ ]:
X

---

# **ML Modelling**

### **KNN Model**

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest, f_classif, VarianceThreshold
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer 
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
no_delta_radiomics = pd.concat([clinic_data_cleaned, a_radiomics, b_radiomics], axis=1)
no_delta_radiomics = no_delta_radiomics.iloc[:-1,:]

In [ ]:
X_with_delta = modelling_data.drop(columns=target_variables + date_related + ['T','E','id_a','id_b','id_cleaned','id']) 

X_without_delta = no_delta_radiomics.drop(columns=target_variables + date_related + ['T', 'E', 'id_a','id_b','id_cleaned'])

In [ ]:
y = modelling_data['E']

In [ ]:
RANDOM_STATE = 42

K_BEST_OPTIONS = [5, 10, 20] # a range of k values to try in the grid search

VAR_THRSH = [0.1, 0.5, 1.0] # variance threshold options to try in grid search

In [ ]:
scale_radio = a_radiomics.columns[1:].tolist() + b_radiomics.columns[1:].tolist() 

In [ ]:
COLUMNS_TO_SCALE_WITH_DELTA = ['scr_age', 'scr_bmi', 'cli_st_trombocytes', 'cli_st_neutrophils','cli_st_ldh', 'cli_st_crp', 'cli_st_ferritin' ] + scale_radio + delta_radiomics_results.columns[1:].tolist()

COLUMNS_TO_SCALE_WITHOUT_DELTA = ['scr_age', 'scr_bmi', 'cli_st_trombocytes', 'cli_st_neutrophils','cli_st_ldh', 'cli_st_crp', 'cli_st_ferritin' ] + scale_radio

COLUMNS_TO_SCALE_NO_RADIO = ['scr_age', 'scr_bmi', 'cli_st_trombocytes', 'cli_st_neutrophils','cli_st_ldh', 'cli_st_crp', 'cli_st_ferritin' ]

COLUMNS_TO_SCALE_ONLY_POINT_A = ['scr_age', 'scr_bmi', 'cli_st_trombocytes', 'cli_st_neutrophils','cli_st_ldh', 'cli_st_crp', 'cli_st_ferritin' ] + a_radiomics.columns[1:].tolist()
COLUMNS_TO_SCALE_ONLY_POINT_B = ['scr_age', 'scr_bmi', 'cli_st_trombocytes', 'cli_st_neutrophils','cli_st_ldh', 'cli_st_crp', 'cli_st_ferritin' ] + b_radiomics.columns[1:].tolist()

In [ ]:
only_clinic = clinic_data_cleaned.drop(columns=target_variables + date_related + ['id_cleaned','E','T'])

In [ ]:
X_with_a_radiomics = pd.concat([only_clinic, a_radiomics], axis=1).iloc[:-1,:]

In [ ]:
X_with_b_radiomics = pd.concat([only_clinic, b_radiomics], axis=1).iloc[:-1,:]

In [ ]:
for name, X, COLUMNS_TO_SCALE in [("with delta radiomics features", X_with_delta, COLUMNS_TO_SCALE_WITH_DELTA), 
                                  ("without delta radiomics features", X_without_delta, COLUMNS_TO_SCALE_WITHOUT_DELTA), 
                                  ("only clinical features", only_clinic, COLUMNS_TO_SCALE_NO_RADIO),
                                  ("only clinical + point A radiomics", X_with_a_radiomics, COLUMNS_TO_SCALE_ONLY_POINT_A),
                                  ("only clinical + point B radiomics", X_with_b_radiomics, COLUMNS_TO_SCALE_ONLY_POINT_B)]:
    print("\n" + "="*20)
    print(f"Starting new modelling run: {name}")
    print("="*20 + "\n")

    x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
    # Define the ColumnTransformer for selective scaling
    preprocessor = ColumnTransformer(
        transformers=[
            # Apply StandardScaler only to the specified list of columns
            ('scaling_pipeline', StandardScaler(), COLUMNS_TO_SCALE)
        ],
        # 'remainder='passthrough' is crucial: it keeps all other columns untouched
        remainder='passthrough' 
    )
    # Step 1: Feature Scaling (Crucial for SVMs)
    scaler = StandardScaler()
    # Step 2: Variance Threshold (New Step)
    # Removes features whose variance is below the threshold.
    # This step helps pre-filter non-informative features before SelectKBest.
    variance_filter = VarianceThreshold()
    # Step 3: Feature Selection
    # Use SelectKBest with f_classif (ANOVA F-value)
    feature_selector = SelectKBest(score_func=f_classif)

    # Step 4: Classifier
    # Using a 'linear' kernel is often more stable than RBF when N is small and P is large.
    # Note: The C parameter will be tuned using GridSearchCV.
    classifier = KNeighborsClassifier()
    #SVC(random_state=RANDOM_STATE)

    # Build the pipeline with the new scaling and threshold steps
    pipeline = Pipeline(steps=[
        ('preprocess', preprocessor),                 # Standardize features first
        ('variance_threshold', variance_filter),
        ('select_kbest', feature_selector), 
        ('classifier', classifier)      
    ])

    print("Pipeline updated, now includes StandardScaler and VarianceThreshold for initial feature filtering.")
    print("-" * 50)


    # --- 3. Hyperparameter Tuning with GridSearchCV ---

    # Define the parameter grid to search over.


    param_grid = {

        'variance_threshold__threshold': VAR_THRSH,
        # Tuning the 'k' parameter of SelectKBest (how many features to select)
        'select_kbest__k': K_BEST_OPTIONS,
        # Tuning the 'C' regularization parameter of the SVC
        'classifier__n_neighbors': [3, 4, 5] 
        }
    
    # Use GridSearchCV with the pipeline and the parameter grid.
    # The inner Cross-Validation (cv=5) ensures stable feature selection.
    # The pipeline ensures feature selection is ONLY fitted on the training folds.
    grid_search = GridSearchCV(
        pipeline, 
        param_grid, 
        cv=5,                 # Use 5-fold cross-validation
        scoring='accuracy',   # Metric to optimize
        n_jobs=-1             # Use all available cores
    )

    print("Starting Grid Search training...")
    grid_search.fit(x_train, y_train)


    # --- 4. Evaluate the Best Model ---

    print("\nGrid Search Complete.")
    print(f"Best parameters found: {grid_search.best_params_}")
    print(f"Best cross-validation score (Training Set): {grid_search.best_score_:.4f}")

    # Predict on the held-out test data using the best estimator found by GridSearchCV
    y_pred = grid_search.predict(x_test)

    # Evaluate performance on the test set
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Test Accuracy (unseen data): {accuracy:.4f}")


    # --- 5. Inspecting the Feature Selection Results of the Best Model ---

    best_selector = grid_search.best_estimator_['select_kbest']
    scores = best_selector.scores_
    p_values = best_selector.pvalues_
    k_best = grid_search.best_params_['select_kbest__k']
    all_features = x_train.columns.tolist()

    # 5.1 Determine the order of features after the ColumnTransformer
    # ColumnTransformer puts the transformed columns first, then the remainder.
    passthrough_features = [col for col in all_features if col not in COLUMNS_TO_SCALE]
    full_feature_names_after_preprocessor = COLUMNS_TO_SCALE + passthrough_features

    # 5.2 Apply the variance mask to the ordered list of feature names
    best_variance_filter = grid_search.best_estimator_['variance_threshold']
    variance_mask = best_variance_filter.get_support()
    features_after_variance_filter = np.array(full_feature_names_after_preprocessor)[variance_mask].tolist()

    # 5.3 Create a DataFrame to sort and display all feature scores from SelectKBest
    feature_ranking = pd.DataFrame({
        'Feature': features_after_variance_filter,
        'F_Score': scores,
        'P_Value': p_values
    })

    # Sort by F_Score (highest first) and print the top K
    feature_ranking = feature_ranking.sort_values(by='F_Score', ascending=False)
    top_k_features = feature_ranking.head(k_best)


    print("\n--- Feature Ranking by SelectKBest (Best Model) ---")
    print(f"Optimal Variance Threshold used: {grid_search.best_params_['variance_threshold__threshold']:.2f}")
    print(f"Number of features remaining after Variance Threshold: {len(features_after_variance_filter)}")
    print(f"Optimal number of features (k) chosen by SelectKBest: {k_best}")

    # Display the top features, scores, and p-values
    print(top_k_features.to_string(float_format="%.4f"))

